In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler

In [4]:
devices = pd.read_csv("devices.csv")
reading_types = pd.read_csv("reading_types.csv")

chunks = []
sample_df = []
for chunk in pd.read_csv('sampled_readings.csv', chunksize=1000):
    chunks.append(chunk)


sample_df = pd.concat(chunks, ignore_index=True)

In [5]:
devices.head(6)

,device_id,building_id
0,1,1
1,2,1
2,3,1
3,4,1
4,43,2
5,44,3


In [6]:
reading_types.head(6)

,reading_type_id,reading_type_name
0,1,CO2
1,2,CO
2,3,TVOC
3,4,PM10
4,5,PM25
5,6,PM01


In [7]:
sample_df.head(10)

,device_id,date,value_type_id,value
0,70,2023-08-11 00:59:58,6,6.0
1,194,2023-03-24 20:01:56,5,0.2
2,134,2023-02-21 23:25:28,6,0.0
3,117,2023-07-28 06:15:20,12,48.5
4,46,2023-01-18 09:50:13,7,15.6
5,120,2023-04-19 03:35:57,11,22.6
6,145,2023-01-30 06:27:30,1,497.0
7,196,2023-10-06 20:59:27,6,1.9
8,191,2023-07-17 13:21:08,4,0.0
9,2,2023-10-19 17:48:49,2,0.7


In [8]:
df = sample_df.merge(devices, on="device_id", how="left")

In [9]:
df.head()

,device_id,date,value_type_id,value,building_id
0,70,2023-08-11 00:59:58,6,6.0,12
1,194,2023-03-24 20:01:56,5,0.2,33
2,134,2023-02-21 23:25:28,6,0.0,25
3,117,2023-07-28 06:15:20,12,48.5,20
4,46,2023-01-18 09:50:13,7,15.6,6


In [10]:
df = pd.merge(df, reading_types, left_on='value_type_id', right_on='reading_type_id')
df.head(5)

,device_id,date,value_type_id,value,building_id,reading_type_id,reading_type_name
0,70,2023-08-11 00:59:58,6,6.0,12,6,PM01
1,134,2023-02-21 23:25:28,6,0.0,25,6,PM01
2,196,2023-10-06 20:59:27,6,1.9,33,6,PM01
3,138,2023-09-24 03:33:50,6,0.0,25,6,PM01
4,47,2023-02-03 20:11:14,6,0.0,6,6,PM01


In [ ]:
df.describe()

,device_id,value_type_id,value,building_id,reading_type_id,date_time
count,3.900422e+07,3.900422e+07,3.900422e+07,3.900422e+07,3.900422e+07,39004224
mean,1.340473e+02,6.297514e+00,7.096466e+01,2.276067e+01,6.297514e+00,2023-06-08 09:11:05.237848064
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2023-01-01 00:00:00
25%,8.000000e+01,3.000000e+00,0.000000e+00,1.200000e+01,3.000000e+00,2023-03-17 03:26:00
50%,1.390000e+02,6.000000e+00,1.030000e+01,2.500000e+01,6.000000e+00,2023-05-13 00:57:00
75%,1.970000e+02,9.000000e+00,2.860000e+01,3.300000e+01,9.000000e+00,2023-08-31 15:27:00
max,2.420000e+02,1.200000e+01,1.999100e+03,4.100000e+01,1.200000e+01,2023-12-31 23:59:00
std,6.772918e+01,3.496428e+00,1.622930e+02,1.173501e+01,3.496428e+00,NaN


In [11]:

df["date_time"] = pd.to_datetime(df["date"])
df["date_time"] = df["date_time"].dt.floor('T')

In [52]:
result= df.groupby([
    pd.Grouper(key='building_id'),
    pd.Grouper(key='reading_type_id'),
    pd.Grouper(key="date_time", freq="5min")])

In [53]:
result_mean = result['value'].mean()
result_var = result['value'].var()

In [54]:
result_mean = result_mean.reset_index()

In [55]:
result_var = result_var.reset_index()

In [56]:
result_var

,building_id,reading_type_id,date_time,value
0,1,1,2023-01-01 00:00:00,206.250000
1,1,1,2023-01-01 00:05:00,450.333333
2,1,1,2023-01-01 00:10:00,364.500000
3,1,1,2023-01-01 00:20:00,68.666667
4,1,1,2023-01-01 00:25:00,84.500000
...,...,...,...,...
11820522,41,12,2023-12-31 23:35:00,4.500000
11820523,41,12,2023-12-31 23:40:00,NaN
11820524,41,12,2023-12-31 23:45:00,NaN
11820525,41,12,2023-12-31 23:50:00,NaN


In [57]:
building_df = pd.merge(result_var, result_mean, on=['building_id', 'reading_type_id', 'date_time'])

In [58]:
building_df

,building_id,reading_type_id,date_time,value_x,value_y
0,1,1,2023-01-01 00:00:00,206.250000,517.750000
1,1,1,2023-01-01 00:05:00,450.333333,525.333333
2,1,1,2023-01-01 00:10:00,364.500000,514.500000
3,1,1,2023-01-01 00:20:00,68.666667,503.000000
4,1,1,2023-01-01 00:25:00,84.500000,522.500000
...,...,...,...,...,...
11820522,41,12,2023-12-31 23:35:00,4.500000,39.900000
11820523,41,12,2023-12-31 23:40:00,NaN,38.300000
11820524,41,12,2023-12-31 23:45:00,NaN,47.100000
11820525,41,12,2023-12-31 23:50:00,NaN,47.000000


In [61]:
df.describe()

,device_id,value_type_id,value,building_id,reading_type_id,date_time
count,3.900422e+07,3.900422e+07,3.900422e+07,3.900422e+07,3.900422e+07,39004224
mean,1.340473e+02,6.297514e+00,7.096466e+01,2.276067e+01,6.297514e+00,2023-06-08 09:11:05.237848064
min,1.000000e+00,1.000000e+00,0.000000e+00,1.000000e+00,1.000000e+00,2023-01-01 00:00:00
25%,8.000000e+01,3.000000e+00,0.000000e+00,1.200000e+01,3.000000e+00,2023-03-17 03:26:00
50%,1.390000e+02,6.000000e+00,1.030000e+01,2.500000e+01,6.000000e+00,2023-05-13 00:57:00
75%,1.970000e+02,9.000000e+00,2.860000e+01,3.300000e+01,9.000000e+00,2023-08-31 15:27:00
max,2.420000e+02,1.200000e+01,1.999100e+03,4.100000e+01,1.200000e+01,2023-12-31 23:59:00
std,6.772918e+01,3.496428e+00,1.622930e+02,1.173501e+01,3.496428e+00,NaN


In [62]:
null_rows = df[df.isnull().any(axis=1)]

if null_rows.empty:
    print('empty')
else:
    print(null_rows.shape)

empty


In [63]:
df.columns

Index(['device_id', 'date', 'value_type_id', 'value', 'building_id',
       'reading_type_id', 'reading_type_name', 'date_time'],
      dtype='object')

In [66]:
df = df[['device_id', 'date', 'value_type_id', 'value', 'building_id']]
df

,device_id,date,value_type_id,value,building_id
0,70,2023-08-11 00:59:58,6,6.0,12
1,134,2023-02-21 23:25:28,6,0.0,25
2,196,2023-10-06 20:59:27,6,1.9,33
3,138,2023-09-24 03:33:50,6,0.0,25
4,47,2023-02-03 20:11:14,6,0.0,6
...,...,...,...,...,...
39004219,69,2023-02-26 14:43:50,9,19.2,12
39004220,142,2023-10-30 12:59:39,9,19.2,25
39004221,185,2023-03-15 13:24:40,9,20.0,33
39004222,194,2023-11-05 22:08:58,9,19.8,33


In [ ]:
from tslearn.clustering import TimeSeriesKMeans
from tslearn.preprocessing import TimeSeriesScalerMeanVariance

# Assuming X is your time series data, where each row represents a time series
# X should be a 3D array with shape (n_samples, n_timestamps, n_features)

# Scale the time series data
scaler = TimeSeriesScalerMeanVariance(mu=0.0, std=1.0)
X_scaled = scaler.fit_transform(X)

# Define the TimeSeriesKMeans clustering model
kmeans = TimeSeriesKMeans(n_clusters=5, metric='DTW')  # Specify the number of clusters and distance metric

# Fit the clustering model to your data
kmeans.fit(X_scaled)

# Optionally, you can use the trained model for prediction or further analysis
